## Installing Required Packages

In [2]:
%pip install tensorflow matplotlib opencv-python mediapipe scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [3]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

2023-08-23 15:40:49.105459: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-23 15:40:49.944558: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


### Setting Up Media Pipe holistic Model to detect and draw keypoints (landmarks)

In [4]:
# MediaPipe Holistic will be used to detect key points
# and draw them on the cv2 feed
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

In [5]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = model.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image, results

In [6]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION,
                              mp_drawing.DrawingSpec(color=(80, 110, 10), thickness=1, circle_radius=1),
                              mp_drawing.DrawingSpec(color=(80, 256, 121), thickness=1, circle_radius=1)
                              )
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                              mp_drawing.DrawingSpec(color=(80, 22, 10), thickness=2, circle_radius=4),
                              mp_drawing.DrawingSpec(color=(80, 44, 121), thickness=2, circle_radius=2)
                              )
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                              mp_drawing.DrawingSpec(color=(121, 22, 76), thickness=2, circle_radius=4),
                              mp_drawing.DrawingSpec(color=(121, 44, 250), thickness=2, circle_radius=2)
                              )
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                              mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=4),
                              mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)
                              )


In [7]:
def extract_keypoints(results):  
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

### Generating and saving data for training and testing

In [8]:
DATA_PATH = os.path.join('MP_DATA')
actions = np.array(['hello', 'thanks', 'iloveyou'])
no_sequences = 30
sequence_length = 30  


# Making 3 folders to store 3 features, each has 30 samples (videos) with each sample containing 30 frames
# with all the landmarks concatenated into a np array

In [9]:
for action in actions:
    for sequence in range(no_sequences):
        try:
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

### Capturing data using Open CV

In [30]:
cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    for action in actions:
        for sequence in range(no_sequences):
            for frame_num in range(sequence_length):
                # Reading frame from webcam (device -> 0)
                ret, frame = cap.read()
                # Make detections of Key Points
                image, results = mediapipe_detection(frame, holistic)
                # Drawing landmarks
                draw_landmarks(image, results)

                # Collecting videos frames
                # accounting for wait (delay)
                if frame_num == 0:
                    cv2.putText(image, 'STARTING COLLECTION', (120, 200),
                                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15, 12),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    cv2.imshow('OpenCV Feed', image)
                    cv2.waitKey(2000)
                else:
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15, 12),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    cv2.imshow('OpenCV Feed', image)

                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)

                # Exiting the loop 
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
    cap.release()
    cv2.destroyAllWindows()

In [10]:
cap.release()
cv2.destroyAllWindows()

NameError: name 'cap' is not defined

### Labeling data with appropriate features

In [11]:
label_map = {label:num for num, label in enumerate(actions)}
print(label_map)

{'hello': 0, 'thanks': 1, 'iloveyou': 2}


In [12]:
sequences, labels = [], []
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [13]:
np.array(labels).shape

(90,)

In [14]:
X = np.array(sequences)

In [15]:
X.shape

(90, 30, 1662)

In [16]:
y = to_categorical(labels).astype(int)

### Splitting data into training and testing

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

In [18]:
X_train.shape

(81, 30, 1662)

In [19]:
len(X_train)

81

## Building the Model

#### Setting Up TensorBoard (log file to be used by TB) to generate realtime logs and analytics on training

In [20]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

# use tensorboard --logdir=. (from /Logs/train dir) on commandline

In [21]:
%load_ext tensorboard

In [22]:
%reload_ext tensorboard
%tensorboard --logdir Logs/train

In [23]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30, 1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))


2023-08-23 15:42:19.418768: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-08-23 15:42:19.486785: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [24]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [25]:
model.fit(X_train, y_train, epochs=2000, callbacks=[tb_callback])

Epoch 1/2000


2023-08-23 15:42:30.429084: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 16154640 exceeds 10% of free system memory.


3/3 [==============================] - 3s 72ms/step - loss: 1.3372 - categorical_accuracy: 0.2469
Epoch 2/2000
3/3 [==============================] - 0s 62ms/step - loss: 1.1762 - categorical_accuracy: 0.3704
Epoch 3/2000
3/3 [==============================] - 0s 59ms/step - loss: 3.2149 - categorical_accuracy: 0.3827
Epoch 4/2000
3/3 [==============================] - 0s 59ms/step - loss: 4.4645 - categorical_accuracy: 0.2716
Epoch 5/2000
3/3 [==============================] - 0s 61ms/step - loss: 1.4834 - categorical_accuracy: 0.3086
Epoch 6/2000
3/3 [==============================] - 0s 60ms/step - loss: 1.2306 - categorical_accuracy: 0.2593
Epoch 7/2000
3/3 [==============================] - 0s 59ms/step - loss: 1.3320 - categorical_accuracy: 0.2963
Epoch 8/2000
3/3 [==============================] - 0s 60ms/step - loss: 1.7223 - categorical_accuracy: 0.2963
Epoch 9/2000
3/3 [==============================] - 0s 59ms/step - loss: 1.1781 - categorical_accuracy: 0.3457
Epoch 10/2000


In [26]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 30, 64)            442112    
                                                                 
 lstm_1 (LSTM)               (None, 30, 128)           98816     
                                                                 
 lstm_2 (LSTM)               (None, 64)                49408     
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 3)                 99        
                                                                 
Total params: 596675 (2.28 MB)
Trainable params: 596675 

In [27]:
# Saving weights
model.save('action.h5')
# to reload weights
# model.load_weights('action.h5)

/home/aryan/Desktop/test/test_env/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [28]:
model.save('action.keras')

### Making Predictions and Evaluating the model

In [29]:
yhat = model.predict(X_test)

1/1 [==============================] - 0s 277ms/step


In [30]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [31]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[4, 1],
        [0, 4]],

       [[5, 1],
        [1, 2]],

       [[7, 0],
        [1, 1]]])

In [32]:
accuracy_score(ytrue, yhat)

0.7777777777777778

In [33]:
# function to render (visualize) probability rectangles dynamically on prediction frames
colors = [(245,117,16), (117,245,16), (16,117,245)]
def probability_visualization(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0, 60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
    return output_frame

### Making real time predictions

In [34]:
sequence = []
sentence = []
predictions = []
threshold = 0.7

cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        # Reading frame from webcam (0)
        ret, frame = cap.read()
        # Make detections of Key Points
        image, results = mediapipe_detection(frame, holistic)
        # Drawing landmarks
        draw_landmarks(image, results)

        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]

        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            predictions.append(np.argmax(res))

            if np.unique(predictions[-10:])[0] == np.argmax(res):
                if res[np.argmax(res)] > threshold:
                    if len(sentence) > 0:
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5:
                sentence = sentence[-5:]
            
            image = probability_visualization(res, actions, image, colors)

        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ''.join(sentence), (3, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

        # Displaying the captured frame (image)
        cv2.imshow('OpenCV Feed', image)
        # Exiting the loop 
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


1/1 [==============================] - 0s 23ms/step
